# 안녕하세요^^ 
## AIVLE 2차 미니프로젝트 '서울시 생활정보 기반 대중교통 수요 분석 ' 과정 2번 파일입니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

## [프로젝트 흐름]

0.튜토리얼에서 주민등록 데이터를 함께 전처리/체험 해 봅니다.

1.서울시 버스 정류장 데이터를 전처리/분석 합니다.

### 2.서울시 유동인구 데이터를 전처리/분석 합니다.

3.서울시 주민등록 데이터를 전처리/분석 합니다.

4.서울시 업종등록 데이터를 전처리/분석 합니다.

5.위 4개 데이터를 합쳐서 버스 노선 추가 필요 대상 지역(구 단위)을 선정합니다.

---

# 데이터 분석부터 먼저 시작해보겠습니다.
## "구 별 유동인구 데이터" 를 확인해 보도록 하겠습니다

In [1]:
# 필요 라이브러리부터 설치합니다.
%pip install pandas seaborn

Note: you may need to restart the kernel to use updated packages.



* 처음에 제공되는 기본/추가 데이터는 'raw_data' 폴더에 있습니다.

[기본 데이터]
* 생활이동_자치구_2022.12_00~24시.csv

* 생활이동_자치구_2022.12_00시.csv 데이터는 버스에서 하차한 시간이 0시 0분 ~ 0시 59분인 경우의 데이터입니다.
* 생활이동_자치구_2022.12_13시.csv 데이터는 버스에서 하차한 시간이 13시 0분 ~ 13시 59분인 경우의 데이터입니다.

[데이터 소개]
*  2022년 12월 서울시 구별 이동인원 데이터

[변수 소개]
* 도착시간 : 0 -> "0:00~0:59"를 의미 
* 출발 자치구 : 자치구 코드
* 성별 : F(여성), M(남성)
* 나이 : 10 ~ 79세 (5세 단위), 0 ~ 10세, 80세 이상으로 구분
* 유형 : H(야간상주지), W(주간상주지), E(기타)
* 평균 이동 시간 : 동일 열 내 이동인구의 월 단위 평균 이동 시간 (10분단위)
* 이동인구 : 추정 합산값으로 소수점 표출, 개인정보 비식별화를 위해 3명 미만의 경우 * 처리

---

# 1.데이터 불러오기
## 모든 미니 프로젝트의 시작은 '데이터 불러오기' 부터라고 할 수 있습니다.
+ KeyPoint : 불러오고자 하는 데이터에 따라 자유롭게 변수로 지정할 수 있다.

###  데이터 프레임을 불러오고 변수로 저장(여기서는 CSV 기준으로 진행)
* csv : pd.read_csv("파일이름. csv")
* txt : pd.read_csv("파일이름. csv", sep="구분자")
* xlsx : pd.read_excel('파일이름.xlsx')
* pickle : pd.read_pickle("파일이름.pkl") <br>
 [참고] pickle은 파이썬의 모든 객체를 파일로 저장할 수 있는 방법으로 DataFrame,List,Dict 등 모든 객체 저장 가능(특히 sklearn라이브러리를 통해 모델을 학습시키고, 저장할 때 많이 사용)

#### [실습문제1] 데이터 로딩
* Pandas 라이브러리를 활용해서 0시~24시 엑셀을 합친 데이터프레임을 만들어볼게요.
    * 주어진 소스코드를 실행해보세요.
    * 데이터 파일 로딩시 참고 사항 
        * 구분자(sep)는 ',' 입니다
        * cp949 인코더를 사용해 주세요

In [62]:
# 아래에 실습코드를 작성하고 결과를 확인합니다. 약 2분 내외의 시간이 소요됩니다.
import pandas as pd
from glob import glob
file_names = glob("rawdata/seoul_moving/*.csv") 
total = pd.DataFrame()

for file_name in file_names:
    temp = pd.read_csv(file_name, sep = ',', encoding='cp949')
    total = pd.concat([total, temp])

In [63]:
# 데이터 프레임의 Shape을 확인합니다.
total.shape

(29558734, 10)

---

# 2.기본 정보 확인 및 클렌징

+ 데이터 클렌징 : 결측치, 이상치 등을 제거하여 데이터 분석 결과가 왜곡 되는 문제를 방지하기 위한 정제 과정

#### [실습문제2] 기본 정보 확인하기
* 'total' 데이터의 정보를 확인해보세요.
* 'describe', 'info', 'head' 등 전부 활용해 보겠습니다.

* 1번 열의 의미
    * " 2022년 12월 일요일에 11010(종로구)에서 11010(종로구)으로 이동하여 0시 ~ 0시 59분 사이에 도착했고, E(기타)에서 H(야간상주지(집))으로 이동한 0 ~ 10세 여성은 총 17.68명이며 이들의 평균 이동시간은 75분이다

In [64]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29558734 entries, 0 to 1016071
Data columns (total 10 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   대상연월         int64 
 1   요일           object
 2   도착시간         int64 
 3   출발 시군구 코드    int64 
 4   도착 시군구 코드    int64 
 5   성별           object
 6   나이           int64 
 7   이동유형         object
 8   평균 이동 시간(분)  int64 
 9   이동인구(합)      object
dtypes: int64(6), object(4)
memory usage: 2.4+ GB


In [65]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
total.describe()

,대상연월,도착시간,출발 시군구 코드,도착 시군구 코드,나이,평균 이동 시간(분)
count,29558734.0,2.955873e+07,2.955873e+07,2.955873e+07,2.955873e+07,2.955873e+07
mean,202212.0,1.367634e+01,1.709611e+04,1.719035e+04,4.253275e+01,9.532969e+01
std,0.0,5.868597e+00,9.023087e+03,9.045166e+03,1.781539e+01,8.109942e+01
min,202212.0,0.000000e+00,1.101000e+04,1.101000e+04,0.000000e+00,1.000000e+00
25%,202212.0,1.000000e+01,1.110000e+04,1.111000e+04,3.000000e+01,4.900000e+01
50%,202212.0,1.400000e+01,1.120000e+04,1.120000e+04,4.000000e+01,7.500000e+01
75%,202212.0,1.800000e+01,3.101400e+04,3.101400e+04,5.500000e+01,1.130000e+02
max,202212.0,2.300000e+01,3.900000e+04,3.900000e+04,8.000000e+01,5.078000e+03


#### [실습문제3] 결측치 처리 ( NaN이 아닌 결측치를 처리 해봅시다)

* '인구이동(합)' column은 이동 인구의 합산인데 object 타입입니다. 

    - 왜 일까요? 

    - Tip! (이동인구 : 추정 합산값으로 소수점 표출, 개인정보 비식별화를 위해 3명 미만의 경우 * 처리)

    - 해당 값을 나름의 기준으로 변경하세요

    - '인구이동(합)' 컬럼의 데이터 타입을 수치형으로 변경하세요 

In [67]:
# 최빈값과 평균값을 알 수 없는 상황이라 중위값으로 대체
total.replace("*", "1.5", inplace=True)

In [69]:
# 이동인구(합) column의 데이터 타입을 바꿔주세요
total = total.astype({'이동인구(합)' : 'float'})


#### [실습문제4] 이동유형, 요일, 성별 변수의 value수를 파악

* 역시나 value_counts()를 활용해 봅시다.

In [71]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
total['이동유형'].value_counts()

EH    5939025
HE    5667459
EE    5260549
WH    3128038
WE    2828357
HW    2730192
EW    2320634
WW    1030463
HH     654017
Name: 이동유형, dtype: int64

In [72]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
total['요일'].value_counts()

금    4706484
토    4705795
목    4441304
화    3982479
수    3955721
월    3945340
일    3821611
Name: 요일, dtype: int64

* 12월의 요일 수를 알려드릴게요
    * 일 : 4, 월 : 4, 화 : 4, 수 : 4, 목 : 5, 금 : 5, 토 : 5

In [73]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

total['성별'].value_counts()


M    16606573
F    12952161
Name: 성별, dtype: int64

#### [실습문제5] 서울 지역의 데이터만 골라 보도록 하겠습니다.

* 우리는 서울시의 데이터만 필요합니다!

* 출발과 도착이 모두 서울인 데이터만 추출해 주세요.

In [74]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
total = total[(total["출발 시군구 코드"] < 20000) & (total["도착 시군구 코드"] < 20000)]
total.head()

,대상연월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202212,일,0,11010,11010,F,0,EE,27,83.82
1,202212,일,0,11010,11010,F,10,EH,75,17.68
2,202212,일,0,11010,11010,F,10,HH,59,11.80
3,202212,일,0,11010,11010,F,10,WW,66,6.00
4,202212,일,0,11010,11010,F,10,EE,96,11.92


# H(야간상주지), W(주간상주지), E(기타)

In [118]:
total_old = total[(total['나이'] >= 65) ]

,대상연월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
89,202212,일,0,11010,11010,F,65,HH,54,4.60
90,202212,일,0,11010,11010,F,65,EH,31,37.18
91,202212,일,0,11010,11010,F,65,WH,35,46.40
92,202212,일,0,11010,11010,F,65,EE,26,4.60
93,202212,일,0,11010,11010,F,70,EE,8,16.92
...,...,...,...,...,...,...,...,...,...,...
968417,202212,토,23,11250,11250,M,80,WW,10,9.52
968418,202212,토,23,11250,11250,M,80,WH,34,19.03
968419,202212,토,23,11250,11250,M,80,HW,55,9.46
968420,202212,토,23,11250,11250,M,80,EH,193,28.29


In [119]:
total_old.groupby(by=['도착 시군구 코드'], as_index = False)['나이'].mean().sort_values(by = '나이', ascending = False).head()

,도착 시군구 코드,나이
0,11010,70.963211
22,11230,70.837600
5,11060,70.823851
21,11220,70.769809
1,11020,70.754979


#### [실습문제6] 서울지역의 구 별로 groupby하여 데이터를 확인하겠습니다 (유동인구 기준)

* 유출이 제일 많은 구는?

* 유입이 제일 많은 구는?

* 유출이 제일 적은 구는?

* 유입이 제일 적은 구는?

* 참고자료 서울 구별 코드
    * 11010	종로구
    * 11020	중구
    * 11030	용산구
    * 11040	성동구
    * 11050	광진구
    * 11060	동대문구
    * 11070	중랑구
    * 11080	성북구
    * 11090	강북구
    * 11100	도봉구
    * 11110	노원구
    * 11120	은평구
    * 11130	서대문구
    * 11140	마포구
    * 11150	양천구
    * 11160	강서구
    * 11170	구로구
    * 11180	금천구
    * 11190	영등포구
    * 11200	동작구
    * 11210	관악구
    * 11220	서초구
    * 11230	강남구
    * 11240	송파구
    * 11250	강동구

In [53]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

#### [실습문제7] '도착 시군구 코드' 를 기준으로 각 구별로 이동인구(합)을 구해보겠습니다

* groupby 사용, as_index=False로 설정해주세요

In [56]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
total_old.groupby(by=['도착 시군구 코드'], as_index = False)['이동인구(합)'].sum().sort_values(by = '이동인구(합)', ascending = False)

,도착 시군구 코드,이동인구(합)
22,11230,6969763.45
23,11240,6541944.51
21,11220,5524416.83
24,11250,4874790.74
10,11110,4549718.94
15,11160,4326566.07
18,11190,4281716.46
0,11010,4008334.12
11,11120,3989304.30
5,11060,3873895.32


In [112]:
total_old.head()

,대상연월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
89,202212,일,0,11010,11010,F,65,HH,54,4.60
90,202212,일,0,11010,11010,F,65,EH,31,37.18
91,202212,일,0,11010,11010,F,65,WH,35,46.40
92,202212,일,0,11010,11010,F,65,EE,26,4.60
93,202212,일,0,11010,11010,F,70,EE,8,16.92


In [114]:
total_old.groupby(by=['나이'], as_index = False)['평균 이동 시간(분)'].mean().sort_values(by = '평균 이동 시간(분)', ascending = False)

,나이,평균 이동 시간(분)
0,65,71.613693
1,70,70.752815
2,75,68.107278
3,80,65.931727


In [102]:
import scipy.stats as spst

# 노인의 나이별 이동인구(합)은?

In [131]:
total_old.groupby(by=['나이'], as_index = False)['이동인구(합)'].sum().sort_values(by = '이동인구(합)', ascending = False)

,나이,이동인구(합)
0,65,34663221.18
1,70,22629336.45
3,80,20206227.48
2,75,17652591.18


# 노인의 요일별 이동인구합은?

In [141]:
total.groupby(by=['요일'], as_index = False)[['이동인구(합)']].sum().sort_values(by = '이동인구(합)', ascending = False)

,요일,이동인구(합)
0,금,92745157.41
1,목,90142324.06
5,토,76926082.67
6,화,73094160.82
3,월,71131120.53
2,수,71122031.19
4,일,51420671.55


# 노인의 나이별 평균 이동 시간은?

In [126]:
total_old.groupby(by=['나이'], as_index = False)['평균 이동 시간(분)'].mean().sort_values(by = '평균 이동 시간(분)', ascending = False)

,나이,평균 이동 시간(분)
0,65,71.613693
1,70,70.752815
2,75,68.107278
3,80,65.931727


# 노인의 이동유형별 평균 나이는??

In [127]:
total_old.groupby(by=['이동유형'], as_index = False)['나이'].mean().sort_values(by = '나이', ascending = False)

,이동유형,나이
1,EH,71.177586
3,HE,71.170848
0,EE,70.824950
5,HW,70.466651
7,WH,70.373422
4,HH,70.349725
2,EW,69.835683
8,WW,69.828205
6,WE,69.809487


# 노인의 이동유형별 평균 시간은??

In [95]:
total_old.groupby(by=['이동유형'], as_index = False)['평균 이동 시간(분)'].mean().sort_values(by = '평균 이동 시간(분)', ascending = False)

,이동유형,평균 이동 시간(분)
1,EH,78.086552
0,EE,75.812437
3,HE,74.544793
6,WE,66.519276
2,EW,65.338423
5,HW,65.172524
7,WH,63.514657
8,WW,58.504944
4,HH,45.217918


#### [실습문제8] '도착 시군구 코드' 를 기준으로 각 구별로 '이동인구(합)' 의 평균 과 합을 구해보겠습니다

* groupby 사용, as_index=False로 설정해주세요

In [85]:
total_old_age

,도착 시군구 코드,평균 이동 시간(분),이동인구(합)
0,11010,63.932830,42.520623
1,11020,63.697186,37.881022
2,11030,65.564365,30.997030
3,11040,63.759333,36.237285
4,11050,66.555506,42.562568
5,11060,66.312300,42.839082
6,11070,72.003225,46.536750
7,11080,65.748653,47.106255
8,11090,72.389314,47.426511
9,11100,75.322909,46.766183


In [84]:
total_old_age = total_old.groupby(by=['도착 시군구 코드'], as_index = False)[['평균 이동 시간(분)','이동인구(합)']].mean()

In [57]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

# 각 구별 이동인구의 평균 정보가 있는 데이터프레임을 만듭니다.
total_old_mean = total_old.groupby(by=['도착 시군구 코드'], as_index = False)[['평균 이동 시간(분)','이동인구(합)']].mean()

# 각 구별 이동인구의 합계 정보가 있는 데이터프레임을 만듭니다.
total_old_sum = total_old.groupby(by=['도착 시군구 코드'], as_index = False)[['평균 이동 시간(분)','이동인구(합)']].sum()

# 컬럼명을 맞게 변경합니다.
total_old_sum.rename(columns={"평균 이동 시간(분)":'총 이동 시간', '이동인구(합)':'총 이동 인구'},inplace=True)

In [60]:
df_total_old = pd.merge(total_old_sum, total_old_mean, how = 'inner', on = '도착 시군구 코드')
df_total_old.head()

,도착 시군구 코드,총 이동 시간,총 이동 인구,평균 이동 시간(분),이동인구(합)
0,11010,6026820,4008334.12,63.932830,42.520623
1,11020,5884346,3499448.78,63.697186,37.881022
2,11030,5163456,2441140.09,65.564365,30.997030
3,11040,5082639,2888691.45,63.759333,36.237285
4,11050,4914392,3142777.44,66.555506,42.562568


In [50]:
# 파일을 합쳐볼까요
df_total_old = pd.merge(total_old_sum, total_old_mean, how = 'inner', on = '도착 시군구 코드')
df_total_old

,도착 시군구 코드,총 이동 시간,총 이동 인구,평균 이동 시간(분),이동인구(합)
0,11010,29797989,21031880.41,60.903702,42.986773
1,11020,29871978,22491167.48,59.909546,45.107011
2,11030,27170933,15702426.81,59.739050,34.523955
3,11040,27556207,18138400.81,59.495957,39.162194
4,11050,27336591,18876549.38,62.520935,43.172154
5,11060,28861672,18143542.26,61.684211,38.777036
6,11070,25502247,14809957.70,65.605698,38.099294
7,11080,28796293,20096899.66,61.968824,43.247971
8,11090,24756229,13182652.17,67.274921,35.823787
9,11100,22523029,11173779.54,69.232395,34.346514


In [59]:
# 해당 데이터프레임을 'df_seoul_moving.csv' 파일로 저장하세요
df_total_old.to_csv('df_seoul_moving_old.csv')

---

# 3.데이터 분석하기
+ KeyPoint : 데이터의 형태를 살펴보고 다양한 분석기법을 통해 모델링에 적합하도록 정제요소를 선별할 수 있다.
  * 데이터들의 패턴 탐색
  * 변수들간의 관계 파악

#### [실습문제10] 가장 버스 이용승객이 많은 요일을 찾아보세요


In [139]:
total.sort_value

,대상연월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202212,일,0,11010,11010,F,0,EE,27,83.82
1,202212,일,0,11010,11010,F,10,EH,75,17.68
2,202212,일,0,11010,11010,F,10,HH,59,11.80
3,202212,일,0,11010,11010,F,10,WW,66,6.00
4,202212,일,0,11010,11010,F,10,EE,96,11.92
...,...,...,...,...,...,...,...,...,...,...
968417,202212,토,23,11250,11250,M,80,WW,10,9.52
968418,202212,토,23,11250,11250,M,80,WH,34,19.03
968419,202212,토,23,11250,11250,M,80,HW,55,9.46
968420,202212,토,23,11250,11250,M,80,EH,193,28.29


In [ ]:
total.groupby(by=['요일'], as_index = False)[['이동인구(합)']].sum()

,대상연월,이동인구(합)
0,202212,5.265815e+08


#### [실습문제11] 데이터 분포 알아보기
* 다양한 변수를 기준으로 데이터를 분석하고 인사이트를 도출해보세요.

In [24]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.


In [25]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

In [26]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

In [27]:
# 위 차트를 통해 알게된 사실을 정리해봅시다.
# 1. 
# 2.
# 3.

---